[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/rag/rag_with_hugging_face_gemma_mongodb.ipynb)
[![View Article](https://img.shields.io/badge/View%20Article-blue)](https://www.mongodb.com/developer/products/atlas/gemma-mongodb-huggingface-rag/)



In [ ]:
!pip install -q pandas pymongo sentence_transformers
!pip install pymongo[srv]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 71.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 90.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [ ]:
import pymongo
import pandas as pd

In [ ]:
# Used to securely store your API key
from google.colab import userdata

In [ ]:
# @title Step 1: Connect to MongoDB
client = pymongo.MongoClient(userdata.get('mongo_url'))  # Update with your MongoDB connection string

In [ ]:
db = client["hoanghamobilenew"]  # Replace with your database name
collection = db['embedding_for_vector_search']

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("keepitreal/vietnamese-sbert")


def get_embedding(text):
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()


In [ ]:
get_embedding("Ngọc nói ok quá")

[0.40576696395874023,
 -0.14440716803073883,
 -0.013739767484366894,
 -0.34371939301490784,
 -0.15080590546131134,
 0.27168405055999756,
 0.12879057228565216,
 -0.4050556719303131,
 0.019007226452231407,
 0.24476270377635956,
 0.5972773432731628,
 -0.09946757555007935,
 -0.5618730783462524,
 0.564607560634613,
 0.14662347733974457,
 -0.3282106816768646,
 0.2179698497056961,
 -0.11953378468751907,
 0.03542126342654228,
 0.05271591246128082,
 -0.3207148611545563,
 -0.1798800230026245,
 0.4997858703136444,
 0.44037890434265137,
 -0.02430054545402527,
 -0.3057686686515808,
 -0.3810969591140747,
 -0.6422277092933655,
 0.23627470433712006,
 0.39994016289711,
 -0.172150656580925,
 0.6366140842437744,
 -0.07981013506650925,
 -0.4337470233440399,
 -0.007528519723564386,
 -0.10586925595998764,
 -0.09617947787046432,
 -0.041440699249506,
 0.04457412287592888,
 -0.44087710976600647,
 -0.10684013366699219,
 0.018003180623054504,
 0.09118179231882095,
 -0.299881249666214,
 0.37405166029930115,
 0.09

In [ ]:
def vector_search(user_query, collection, limit=4):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."
    
    vector_search_stage = {
        "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 400,
            "limit": limit,
        }
    }

    unset_stage = {
        "$unset": "embedding"
    }

    project_stage = {
        "$project": {
            "_id": 0,
            "title": 1,
            # "product_specs": 1,
            "color_options": 1,
            "current_price": 1,
            "product_promotion": 1,
            "score": {
                "$meta": "vectorSearchScore"
            }
        }
    }

    pipeline = [vector_search_stage, unset_stage, project_stage]

    # Execute the search
    results = collection.aggregate(pipeline)

    return list(results)

In [ ]:
def get_search_result(query, collection):
    get_knowledge = vector_search(query, collection, 10)
    search_result = ""
    i = 0
    for result in get_knowledge:
        print(result)
        if result.get('current_price'):
            i += 1
            search_result += f"\n {i}) Tên: {result.get('title')}"

            if result.get('current_price'):
                search_result += f", Giá: {result.get('current_price')}"
            else:
                # Mock up data
                # Retrieval model pricing from the internet.
                search_result += f", Giá: Liên hệ để trao đổi thêm!"

            if result.get('product_promotion'):
                search_result += f", Ưu đãi: {result.get('product_promotion')}"
    return search_result

In [ ]:
# Conduct query with retrival of sources
query = "Samsung Galaxy Z Fold4 512GB"
source_information = get_search_result(query.lower(), collection).replace('<br>', '\n')
source_information

{'title': 'samsung galaxy z fold4 - 512gb -chính hãng', 'product_promotion': '', 'current_price': '', 'color_options': [], 'score': 0.978950023651123}
{'title': 'samsung galaxy z fold4 - 256gb - chính hãng', 'product_promotion': '', 'current_price': '', 'color_options': [], 'score': 0.9225382208824158}
{'title': 'điện thoại samsung galaxy z fold5 12gb/512gb - chính hãng', 'product_promotion': '- KM 1<br>- Tặng gói Samsung care+ 6 tháng<br>- KM 2<br>- Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+.<br>', 'current_price': '30,990,000 ₫', 'color_options': ['Xanh', 'Màu Đen', 'Kem'], 'score': 0.8769148588180542}
{'title': 'samsung galaxy z flip4 - 256gb - chính hãng', 'product_promotion': '', 'current_price': '', 'color_options': [], 'score': 0.8767529726028442}
{'title': 'điện thoại ai - samsung galaxy s24 - 8gb/512gb - chính hãng', 'product_promotion': '- KM 1<br>- Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+.<br>- KM 2<br>- Giảm 

'\n 1) Tên: điện thoại samsung galaxy z fold5 12gb/512gb - chính hãng, Giá: 30,990,000 ₫, Ưu đãi: - KM 1\n- Tặng gói Samsung care+ 6 tháng\n- KM 2\n- Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+.\n\n 2) Tên: điện thoại ai - samsung galaxy s24 - 8gb/512gb - chính hãng, Giá: 25,490,000 ₫, Ưu đãi: - KM 1\n- Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+.\n- KM 2\n- Giảm thêm 1.000.000đ cho khách hàng thân thiết (Chi tiết LH 1900 2091)\n\n 3) Tên: điện thoại samsung galaxy s23 ultra 12gb/512gb - chính hãng, Giá: 26,490,000 ₫, Ưu đãi: - KM 1\n- Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+.\n\n 4) Tên: điện thoại  samsung galaxy z flip5 8gb/512gb - chính hãng, Giá: 18,990,000 ₫, Ưu đãi: - KM 1\n- Tặng gói Samsung care+ 6 tháng\n- KM 2\n- Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+.\n- KM 3\n- Giảm thêm 1.000.000đ cho khách hàng thân thiết (Chi tiết LH 1900 2091)\n\n 5) Tên: 

In [ ]:
combined_information = f"Hãy trở thành chuyên gia tư vấn bán hàng cho một cửa hàng điện thoại. Câu hỏi của khách hàng: {query}\nTrả lời câu hỏi dựa vào các thông tin sản phẩm dưới đây: {source_information}."
print('==== LLMs Inference')
print(combined_information)

==== LLMs Inference
Hãy trở thành chuyên gia tư vấn bán hàng cho một cửa hàng điện thoại. Câu hỏi của khách hàng: Samsung Galaxy Z Fold4 512GB
Trả lời câu hỏi dựa vào các thông tin sản phẩm dưới đây: 
 1) Tên: điện thoại samsung galaxy z fold5 12gb/512gb - chính hãng, Giá: 30,990,000 ₫, Ưu đãi: - KM 1
- Tặng gói Samsung care+ 6 tháng
- KM 2
- Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+.

 2) Tên: điện thoại ai - samsung galaxy s24 - 8gb/512gb - chính hãng, Giá: 25,490,000 ₫, Ưu đãi: - KM 1
- Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+.
- KM 2
- Giảm thêm 1.000.000đ cho khách hàng thân thiết (Chi tiết LH 1900 2091)

 3) Tên: điện thoại samsung galaxy s23 ultra 12gb/512gb - chính hãng, Giá: 26,490,000 ₫, Ưu đãi: - KM 1
- Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+.

 4) Tên: điện thoại  samsung galaxy z flip5 8gb/512gb - chính hãng, Giá: 18,990,000 ₫, Ưu đãi: - KM 1
- Tặng gói Samsung care+ 6 thá

Gemini

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
genai.configure(api_key=userdata.get('api_key'))

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro')

In [ ]:
%%time
response = model.generate_content(combined_information)
to_markdown(response.text)

CPU times: user 73 ms, sys: 9.8 ms, total: 82.8 ms
Wall time: 10.5 s


> Chào anh/chị, anh/chị đang quan tâm đến Samsung Galaxy Z Fold4 512GB đúng không ạ? Hiện tại bên em chưa có dòng Z Fold4 mà chỉ mới về dòng Z Fold5 và Z Flip5 thôi anh/chị. 
> 
> Anh/chị có muốn tham khảo qua Z Fold5 không ạ? Z Fold5 là phiên bản kế nhiệm của Z Fold4 với nhiều cải tiến vượt trội như: 
> 
> * **Bản lề mới:** Giúp gập máy mượt mà hơn, không còn khoảng trống giữa 2 màn hình khi gập lại, hạn chế bụi bẩn.
> * **Cấu hình mạnh mẽ hơn:** Chip Snapdragon 8 Gen 2 for Galaxy cho hiệu năng vượt trội so với Z Fold4, xử lý mọi tác vụ nặng, chơi game mượt mà.
> * **Camera được nâng cấp:** Cảm biến chính 50MP cho ảnh chụp sắc nét, đẹp mắt hơn.
> 
> Hơn nữa, khi mua Z Fold5 12GB/512GB tại cửa hàng, anh/chị còn nhận được ưu đãi cực kỳ hấp dẫn:
> 
> * **Tặng gói Samsung Care+ 6 tháng:** Bảo vệ máy an toàn khỏi những rủi ro hư hỏng bất ngờ.
> * **Trả góp 0% lãi suất:** Giúp anh/chị dễ dàng sở hữu siêu phẩm với chi phí ban đầu hợp lý.
> 
> Anh/chị có muốn em tư vấn thêm về Z Fold5 hoặc Z Flip5 không ạ? 


Gemma

In [ ]:
from huggingface_hub import login

# Step 1: Login using your API token
api_token = userdata.get('hugging_face_token')  # Replace with your actual API token
login(api_token)

# Now you can interact with the Hugging Face Hub


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Moving tensors to GPU
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=100)
print(tokenizer.decode(response[0]))

<bos>Hãy trở thành chuyên gia tư vấn bán hàng cho một cửa hàng điện thoại. Câu hỏi của khách hàng: Samsung Galaxy Z Fold4 512GB
Trả lời câu hỏi dựa vào các thông tin sản phẩm dưới đây: .

**Samsung Galaxy Z Fold4 512GB**

* Cấu hình: 2680 x 1752 x 125 mm
* Màn hình: Super AMOLED 2X Infinity Display 6.8 inch
* Chipset: Snapdragon 8 Gen 2
* RAM: 12GB
* Battery: Li-Ion 4000 mAh
* Nút lệnh: Bixby, Face Unlock, Google Assistant
